In [ ]:
!pip3 install openai python-dotenv numpy pandas matplotlib seaborn textblob nltk

In [ ]:
!sudo apt install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (373 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [ ]:
!sudo apt update
!sudo apt install build-essential

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,809 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,115 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,571 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.ne

In [ ]:
!pip install pipwin
!pip install pyaudio

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8770 sha256=97054f2f47e27b9324dccb2259e0d24136e2377c6b7d36f3978276dbd105debf
  Stored in directory: /root/.cache/pip/wheels/07/45/2e/51ee464b7c3407327439df34c2fc62f978a1e5084aaf23d201
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=493dc020ebb17eb997906f05571c9952678c5580b593330b0d439bee2dfd1c8f
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25982 sha256=f686da40be0c8b0637b85abd36e3bba8be19c38c7c37fdf1fd419dea65c8973c
  Stored in directory: /root/.cache/pip/wheels/a5/9a/30/1003e89ab4555b81840ca46d361bf184f1e6ad880cae3b62a9
Successfully b

In [ ]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import nltk
from collections import Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import pyaudio
import wave
import threading
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, Output, VBox, HBox, Label
from ipywidgets import widgets

In [ ]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

In [ ]:
load_dotenv()

from google.colab import userdata
userdata.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = userdata.get('OPENAI_API_KEY')
)

def test_api_connection():
  try:
    response = client.models.list()
    print("Successfully connected to OpenAI API!")
    return True
  except Exception as e:
    print(f"Error connecting to OpenAI API: {e}")
    return False

test_api_connection()

Successfully connected to OpenAI API!


True

In [ ]:
class AudioRecorder:
  def __init__(self):
    self.audio_format = pyaudio.paInt16
    self.channels = 1
    self.rate = 16000
    self.chunk = 1024
    self.recording = False
    self.frames = []
    self.audio = pyaudio.PyAudio()

  def start_recording(self):
    self.recording = True
    self.frames = []
    stream = self.audio.open(
        format = self.audio_format,
        channels = self.channels,
        rate = self.rate,
        input = True,
        frames_per_buffer=self.chunk
    )

    print("Recording Started.... Click 'Stop Recording' when finished")

    while self.recording:
      data = stream.read(self.chunk)
      self.frames.append(data)
    stream.stop_stream()
    stream.close()
    print("Recording Stopped !")

  def stop_recording(self):
    self.recording = False

  def save_audio(self, filename="user_response.wav"):
    if not self.frames:
      print("No audio recorded")
      return None

    wave_file = wave.open(filename, 'wb')
    wave_file.setnchannels(self.channels)
    wave_file.setsampwidth(self.audio.get_sample_size(self.audio_format))
    wave_file.setframerate(self.rate)
    wave_file.writeframes(b''.join(self.frames))
    wave_file.close()

    print(f"Audio Saved as : {filename}")
    return filename

  def cleanup(self):
    self.audio.terminate()

audio_recorder = AudioRecorder()

In [ ]:
class IELTSRealTimeEvaluator:
    def __init__(self, client):
        self.client = client
        self.evaluation_history = []

    def transcribe_audio(self, audio_file_path):
        """Transcribe audio using OpenAI Whisper"""
        try:
            with open(audio_file_path, "rb") as audio_file:
                transcript = self.client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    response_format="verbose_json",
                    language="en"
                )

            return {
                "text": transcript.text,
                "duration": transcript.duration,
                "language": transcript.language
            }
        except Exception as e:
            print(f"Error in transcription: {e}")
            return None

    def analyze_speech_with_ai(self, transcription_text, test_part=1):
        """Use OpenAI to analyze speech and provide IELTS evaluation"""

        prompt = f"""
        You are an expert IELTS speaking examiner. Analyze the following response from a test-taker and provide a comprehensive evaluation based on the four IELTS speaking criteria:

        TEST PART: {test_part}
        RESPONSE: "{transcription_text}"

        Provide your evaluation in the following JSON format:
        {{
            "scores": {{
                "fluency_coherence": [score out of 9],
                "lexical_resource": [score out of 9],
                "grammatical_range": [score out of 9],
                "pronunciation": [score out of 9],
                "overall": [average score out of 9]
            }},
            "detailed_analysis": {{
                "fluency_coherence": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "lexical_resource": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "grammatical_range": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "pronunciation": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }}
            }},
            "recommendations": [
                "specific recommendation 1",
                "specific recommendation 2",
                "specific recommendation 3"
            ],
            "band_descriptor": "[explanation of overall band level]"
        }}

        Be thorough and specific in your analysis. Consider the test part when evaluating appropriateness of response.
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an expert IELTS speaking examiner with years of experience evaluating test-takers."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )

            analysis = json.loads(response.choices[0].message.content)
            return analysis

        except Exception as e:
            print(f"Error in AI analysis: {e}")
            return None

    def generate_follow_up_questions(self, transcription_text, test_part=1):
        """Generate follow-up questions based on user's response"""

        prompt = f"""
        Based on the following IELTS test response, generate 3 appropriate follow-up questions for Part {test_part}:

        Response: "{transcription_text}"

        Generate questions that:
        1. Are appropriate for IELTS Speaking Part {test_part}
        2. Build upon the candidate's response
        3. Encourage extended speaking
        4. Are at an appropriate difficulty level

        Return as a JSON array of questions:
        ["question 1", "question 2", "question 3"]
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an IELTS speaking examiner creating follow-up questions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5
            )

            questions = json.loads(response.choices[0].message.content)
            return questions

        except Exception as e:
            print(f"Error generating questions: {e}")
            return ["Could you tell me more about that?", "How do you feel about this topic?", "What's your opinion on this matter?"]

    def generate_audio_feedback(self, feedback_text, voice="alloy"):
        """Generate audio feedback using OpenAI TTS"""
        try:
            response = self.client.audio.speech.create(
                model="tts-1",
                voice=voice,
                input=feedback_text
            )

            audio_filename = f"feedback_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp3"
            with open(audio_filename, "wb") as f:
                f.write(response.content)

            return audio_filename

        except Exception as e:
            print(f"Error generating audio feedback: {e}")
            return None

evaluator = IELTSRealTimeEvaluator(client)


In [ ]:
import threading
import random
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import VBox, HBox, Button, Output, HTML
from IPython.display import clear_output

class IELTSInteractiveInterface:
  def __init__(self, evaluator, audio_recorder):
    self.evaluator = evaluator
    self.audio_recorder = audio_recorder
    self.current_test_part = 1
    self.current_question = ""
    self.recording_thread = None
    self.setup_interface()

  def setup_interface(self):
        """Set up the interactive interface"""
        self.header = widgets.HTML("""
        <div style="text-align: center; background: linear-gradient(45deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px;">
            <h1>🎤 IELTS Speaking Evaluation System</h1>
            <p>Speak directly into your microphone and receive instant AI-powered feedback</p>
        </div>
        """)

        self.part_selector = widgets.Dropdown(
            options=[('Part 1: Introduction & Interview', 1),
                    ('Part 2: Individual Long Turn', 2),
                    ('Part 3: Two-way Discussion', 3)],
            value=1,
            description='Test Part:',
            style={'description_width': 'initial'}
        )

        self.question_display = widgets.HTML("""
        <div style="background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>🎯 Your Question:</h3>
            <p>Click "Get Question" to receive a practice question for your selected test part.</p>
        </div>
        """)

        self.get_question_btn = widgets.Button(description="Get Question", button_style='info', icon='question')
        self.start_recording_btn = widgets.Button(description="Start Recording", button_style='success', icon='microphone')
        self.stop_recording_btn = widgets.Button(description="Stop Recording", button_style='danger', icon='stop', disabled=True)
        self.evaluate_btn = widgets.Button(description="Get Evaluation", button_style='warning', icon='check', disabled=True)

        self.results_output = widgets.Output()

        self.get_question_btn.on_click(self.get_question)
        self.start_recording_btn.on_click(self.start_recording)
        self.stop_recording_btn.on_click(self.stop_recording)
        self.evaluate_btn.on_click(self.evaluate_response)

        self.interface = VBox([
            self.header,
            self.part_selector,
            self.question_display,
            HBox([self.get_question_btn]),
            HBox([self.start_recording_btn, self.stop_recording_btn, self.evaluate_btn]),
            self.results_output
        ])

  def get_question(self, button):
        """Generate a question for the selected test part"""
        self.current_test_part = self.part_selector.value

        questions = {
            1: [
                "Tell me about your hometown. What do you like most about it?",
                "Do you work or study? What do you enjoy most about your work/studies?",
                "What are your hobbies? How long have you been interested in them?",
                "Describe your daily routine. What's your favorite part of the day?",
                "Tell me about your family. Are you close to your family members?"
            ],
            2: [
                "Describe a memorable event from your childhood. You should say: what the event was, when it happened, who was involved, and explain why it was memorable.",
                "Talk about a person who has influenced you. You should say: who this person is, how you know them, what influence they had on you, and explain why this person is important to you.",
                "Describe a place you would like to visit. You should say: where it is, what you can do there, what it looks like, and explain why you want to visit this place.",
                "Talk about a skill you would like to learn. You should say: what the skill is, why you want to learn it, how you would learn it, and explain how this skill would be useful to you."
            ],
            3: [
                "What role does technology play in modern education?",
                "How do you think social media affects relationships between people?",
                "What are the advantages and disadvantages of living in a big city?",
                "How important is it for people to learn about their cultural heritage?",
                "Do you think traditional skills are being lost in modern society? Why or why not?"
            ]
        }

        self.current_question = random.choice(questions[self.current_test_part])

        self.question_display.value = f"""
        <div style="background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>🎯 Your Question (Part {self.current_test_part}):</h3>
            <p style="font-size: 16px; font-weight: bold;">{self.current_question}</p>
            <p style="font-style: italic; color: #666;">
                {'Take a moment to think, then click "Start Recording" and speak for 1-2 minutes.' if self.current_test_part == 1
                else 'You have 1 minute to prepare, then speak for 1-2 minutes.' if self.current_test_part == 2
                else 'Think about your response and speak for 1-2 minutes.'}
            </p>
        </div>
        """

  def start_recording(self, button):
    self.start_recording_btn.disabled = True
    self.stop_recording_btn.disabled = False
    self.evaluate_btn.disabled = True

    with self.results_output:
      clear_output(wait=True)
      print("Recording Started! Speak for 1-2 minutes...")
      print("Click 'Stop Recording' when finished")
    self.recording_thread = threading.Thread(target = self.audio_recorder.start_recording)
    self.recording_thread.start()

  def stop_recording(self, button):
    self.audio_recorder.stop_recording()

    if(self.recording_thread):
      self.recording_thread.join()

    self.start_recording_btn.disabled = False
    self.stop_recording_btn.disabled = True
    self.evaluate_btn.disabled = False

    with self.results_output:
      clear_output(wait=True)
      print("Recording Stopped !")
      print("Click 'Get Evaluation' to analyze your response.")

  def evaluate_response(self, button):
    self.evaluate_btn.disabled = True

    with self.results_output:
      clear_output()
      print("Processing your response !")

      audio_file = self.audio_recorder.save_audio()
      if not audio_file:
        print("No audio recorded. Evaluation failed.")
        self.evaluate_btn.disabled = False
        return

      print("Transcribing your speech...")
      transcription = self.evaluator.transcribe_audio(audio_file)
      if not transcription:
        print("Failed to transcribe audio")
        self.evaluate_btn.disabled = False
        return

      print(f"Transcription completed ({transcription['duration']:.1f})s")

      print("Analyze your response with AI")

      analysis = self.evaluator.analyze_speech_with_ai(transcription['text'], self.current_test_part)

      if not analysis:
        print("Failed to analyze speech !")
        self.evaluate_btn.disabled = False
        return

      self.display_evaluation_results(transcription, analysis)

      follow_ups = self.evaluator.generate_follow_up_questions(transcription['text'], self.current_test_part)
      self.display_follow_up_questions(follow_ups)

    self.evaluate_btn.disabled = False


  def display_evaluation_results(self, transcription, analysis):
    """Display comprehensive evaluation results"""
    with self.results_output:
      print("\n" + "="*60)
      print("YOUR RESPONSE TRANSCRIPTION")
      print("="*60)
      print(f'"{transcription["text"]}"')
      print(f"\nDuration: {transcription['duration']:.1f} seconds")

      print("\n" + "="*60)
      print("IELTS SCORES")
      print("="*60)
      scores = analysis['scores']
      print(f"Overall Score: {scores['overall']}/9")
      print(f"├── Fluency & Coherence: {scores['fluency_coherence']}/9")
      print(f"├── Lexical Resource: {scores['lexical_resource']}/9")
      print(f"├── Grammatical Range: {scores['grammatical_range']}/9")
      print(f"└── Pronunciation: {scores['pronunciation']}/9")

      print(f"\n Band Level : {analysis['band_description']}")
      print("\n" + "="*60)
      print("DETAILED ANALYSIS")
      print("="*60)

      criteria = ['fluency_coherence', 'lexical_resource', 'grammatical_range', 'pronounciation']
      criteria_names = ['Fluency & Coherence', 'Lexical Resource', 'Grammatical Range', 'Pronunciation']

      for criterion, name in zip(criteria, criteria_names):
        details = analysis['detailed_analysis'][criterion]
        print(f"\n{name.upper()}")
        print(f"Assessment: {details['assessment']}")
        print("Strengths:")
        for strength in details['strengths']:
          print(f" - {strength}")

        print("Areas of improvement : ")
        for weakness in details['weaknesses']:
          print(f" - {weakness}")

      print("\n" + "="*60)
      print("RECOMMENDATIONS")
      print("="*60)
      for i, rec in enumerate(analysis['recommendations'], 1):
        print(f"{i}. {rec}")

      self.plot_scores(scores)

  def display_follow_up_questions(self, questions):
        """Display follow-up questions"""
        with self.results_output:
            print("\n" + "="*60)
            print("❓ FOLLOW-UP QUESTIONS")
            print("="*60)
            print("Try answering these questions to practice further:")
            for i, question in enumerate(questions, 1):
                print(f"{i}. {question}")

  def plot_scores(self, scores):
      """Create a visualization of the scores"""
      fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

      # Bar chart of individual scores
      criteria = ['Fluency &\nCoherence', 'Lexical\nResource', 'Grammatical\nRange', 'Pronunciation']
      score_values = [scores['fluency_coherence'], scores['lexical_resource'],
                      scores['grammatical_range'], scores['pronunciation']]

      bars = ax1.bar(criteria, score_values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A'])
      ax1.set_ylim(0, 9)
      ax1.set_ylabel('Score')
      ax1.set_title('IELTS Speaking Scores by Criteria')
      ax1.grid(axis='y', alpha=0.3)

      # Add value labels on bars
      for bar, value in zip(bars, score_values):
          height = bar.get_height()
          ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                  f'{value}', ha='center', va='bottom', fontweight='bold')

      # Pie chart showing overall performance
      overall_score = scores['overall']
      remaining = 9 - overall_score

      ax2.pie([overall_score, remaining], labels=['Achieved', 'Remaining'],
              colors=['#4ECDC4', '#E8E8E8'], startangle=90,
              autopct=lambda pct: f'{pct/100*9:.1f}' if pct > 0 else '')
      ax2.set_title(f'Overall Score: {overall_score}/9')

      plt.tight_layout()
      plt.show()

  def display(self):
      """Display the interface"""
      return self.interface

# Initialize and display the interface
interface = IELTSInteractiveInterface(evaluator, audio_recorder)
interface.display()

### COLAB SPECIFIC BROWSER RECORDING

In [ ]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import nltk
from collections import Counter
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import threading
import random
from IPython.display import display, HTML, clear_output, Javascript
from ipywidgets import Button, Output, VBox, HBox, Label
from ipywidgets import widgets
import io
import base64

# For Google Colab audio recording
from google.colab import output
from google.colab.output import eval_js

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

load_dotenv()

from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)

def test_api_connection():
    try:
        response = client.models.list()
        print("✅ Successfully connected to OpenAI API!")
        return True
    except Exception as e:
        print(f"❌ Error connecting to OpenAI API: {e}")
        return False

test_api_connection()

class AudioRecorder:
    def __init__(self):
        self.recording = False
        self.audio_data = None
        self.is_recording = False

    def start_recording(self):
        """Start recording using Google Colab's built-in audio recording"""
        try:
            print("🎤 Starting recording... Please allow microphone access if prompted.")

            # JavaScript code to record audio in Google Colab
            js_code = """
            const sleep = time => new Promise(resolve => setTimeout(resolve, time))
            const b2text = blob => new Promise(resolve => {
              const reader = new FileReader()
              reader.onloadend = e => resolve(e.srcElement.result)
              reader.readAsDataURL(blob)
            })

            var log = console.log
            var recorder = {}
            var recording = false

            async function record() {
              if (recording) {
                return
              }

              recording = true

              const stream = await navigator.mediaDevices.getUserMedia({audio: true})
              const mediaRecorder = new MediaRecorder(stream)
              const chunks = []

              mediaRecorder.addEventListener('dataavailable', event => {
                chunks.push(event.data)
              })

              mediaRecorder.addEventListener('stop', () => {
                const blob = new Blob(chunks, {type: 'audio/wav'})
                const url = URL.createObjectURL(blob)
                const audio = document.createElement('audio')
                audio.controls = true
                audio.src = url
                document.body.appendChild(audio)

                b2text(blob).then(text => {
                  const data = text.split(',')[1]
                  recorder.data = data
                  recorder.blob = blob
                })

                stream.getTracks().forEach(track => track.stop())
                recording = false
              })

              recorder.mediaRecorder = mediaRecorder
              mediaRecorder.start()
              return mediaRecorder
            }

            async function stop() {
              if (recorder.mediaRecorder && recording) {
                recorder.mediaRecorder.stop()
                await sleep(1000)
                return recorder.data
              }
              return null
            }

            // Start recording
            record()
            """

            display(Javascript(js_code))
            self.is_recording = True
            self.recording = True

        except Exception as e:
            print(f"❌ Error starting recording: {e}")
            self.recording = False

    def stop_recording(self):
        """Stop recording and get the audio data"""
        if not self.is_recording:
            print("❌ No recording in progress")
            return None

        try:
            print("🔴 Stopping recording...")

            # JavaScript to stop recording and get data
            js_code = """
            stop().then(data => {
              if (data) {
                google.colab.kernel.invokeFunction('notebook.set_audio_data', [data], {})
              }
            })
            """

            display(Javascript(js_code))
            self.recording = False
            self.is_recording = False

            # Wait a moment for the data to be processed
            time.sleep(2)

            return True

        except Exception as e:
            print(f"❌ Error stopping recording: {e}")
            self.recording = False
            return False

    def save_audio(self, filename="user_response.wav"):
        """Save the recorded audio"""
        if not hasattr(self, 'audio_data') or self.audio_data is None:
            print("❌ No audio data available to save")
            return None

        try:
            # Decode base64 audio data
            audio_bytes = base64.b64decode(self.audio_data)

            with open(filename, 'wb') as f:
                f.write(audio_bytes)

            print(f"✅ Audio saved as: {filename}")
            return filename

        except Exception as e:
            print(f"❌ Error saving audio: {e}")
            return None

# Create a global function to receive audio data from JavaScript
def set_audio_data(data):
    global audio_recorder
    audio_recorder.audio_data = data
    print("✅ Audio data received")

# Register the function with Colab
output.register_callback('notebook.set_audio_data', set_audio_data)

class IELTSRealTimeEvaluator:
    def __init__(self, client):
        self.client = client
        self.evaluation_history = []

    def transcribe_audio(self, audio_file_path):
        """Transcribe audio using OpenAI Whisper"""
        try:
            print(f"📝 Transcribing audio file: {audio_file_path}")

            with open(audio_file_path, "rb") as audio_file:
                transcript = self.client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    response_format="verbose_json",
                    language="en"
                )

            print("✅ Transcription completed successfully")
            return {
                "text": transcript.text,
                "duration": transcript.duration,
                "language": transcript.language
            }
        except Exception as e:
            print(f"❌ Error in transcription: {e}")
            return None

    def analyze_speech_with_ai(self, transcription_text, test_part=1):
        """Use OpenAI to analyze speech and provide IELTS evaluation"""

        prompt = f"""
        You are an expert IELTS speaking examiner. Analyze the following response from a test-taker and provide a comprehensive evaluation based on the four IELTS speaking criteria:

        TEST PART: {test_part}
        RESPONSE: "{transcription_text}"

        Provide your evaluation in the following JSON format:
        {{
            "scores": {{
                "fluency_coherence": [score out of 9],
                "lexical_resource": [score out of 9],
                "grammatical_range": [score out of 9],
                "pronunciation": [score out of 9],
                "overall": [average score out of 9]
            }},
            "detailed_analysis": {{
                "fluency_coherence": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "lexical_resource": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "grammatical_range": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }},
                "pronunciation": {{
                    "assessment": "[detailed assessment]",
                    "strengths": ["strength 1", "strength 2"],
                    "weaknesses": ["weakness 1", "weakness 2"]
                }}
            }},
            "recommendations": [
                "specific recommendation 1",
                "specific recommendation 2",
                "specific recommendation 3"
            ],
            "band_descriptor": "[explanation of overall band level]"
        }}

        Be thorough and specific in your analysis. Consider the test part when evaluating appropriateness of response.
        """

        try:
            print("🤖 Analyzing speech with AI...")
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an expert IELTS speaking examiner with years of experience evaluating test-takers."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3
            )

            analysis = json.loads(response.choices[0].message.content)
            print("✅ AI analysis completed")
            return analysis

        except Exception as e:
            print(f"❌ Error in AI analysis: {e}")
            return None

    def generate_follow_up_questions(self, transcription_text, test_part=1):
        """Generate follow-up questions based on user's response"""

        prompt = f"""
        Based on the following IELTS test response, generate 3 appropriate follow-up questions for Part {test_part}:

        Response: "{transcription_text}"

        Generate questions that:
        1. Are appropriate for IELTS Speaking Part {test_part}
        2. Build upon the candidate's response
        3. Encourage extended speaking
        4. Are at an appropriate difficulty level

        Return as a JSON array of questions:
        ["question 1", "question 2", "question 3"]
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an IELTS speaking examiner creating follow-up questions."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.5
            )

            questions = json.loads(response.choices[0].message.content)
            return questions

        except Exception as e:
            print(f"❌ Error generating questions: {e}")
            return ["Could you tell me more about that?", "How do you feel about this topic?", "What's your opinion on this matter?"]

class IELTSInteractiveInterface:
    def __init__(self, evaluator, audio_recorder):
        self.evaluator = evaluator
        self.audio_recorder = audio_recorder
        self.current_test_part = 1
        self.current_question = ""
        self.setup_interface()

    def setup_interface(self):
        """Set up the interactive interface"""
        self.header = widgets.HTML("""
        <div style="text-align: center; background: linear-gradient(45deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px; margin-bottom: 20px;">
            <h1>🎤 IELTS Speaking Evaluation System</h1>
            <p>Speak directly into your microphone and receive instant AI-powered feedback</p>
        </div>
        """)

        self.part_selector = widgets.Dropdown(
            options=[('Part 1: Introduction & Interview', 1),
                    ('Part 2: Individual Long Turn', 2),
                    ('Part 3: Two-way Discussion', 3)],
            value=1,
            description='Test Part:',
            style={'description_width': 'initial'}
        )

        self.question_display = widgets.HTML("""
        <div style="background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>🎯 Your Question:</h3>
            <p>Click "Get Question" to receive a practice question for your selected test part.</p>
        </div>
        """)

        self.get_question_btn = widgets.Button(description="Get Question", button_style='info', icon='question')
        self.start_recording_btn = widgets.Button(description="Start Recording", button_style='success', icon='microphone')
        self.stop_recording_btn = widgets.Button(description="Stop Recording", button_style='danger', icon='stop', disabled=True)
        self.evaluate_btn = widgets.Button(description="Get Evaluation", button_style='warning', icon='check', disabled=True)

        self.results_output = widgets.Output()

        self.get_question_btn.on_click(self.get_question)
        self.start_recording_btn.on_click(self.start_recording)
        self.stop_recording_btn.on_click(self.stop_recording)
        self.evaluate_btn.on_click(self.evaluate_response)

        self.interface = VBox([
            self.header,
            self.part_selector,
            self.question_display,
            HBox([self.get_question_btn]),
            HBox([self.start_recording_btn, self.stop_recording_btn, self.evaluate_btn]),
            self.results_output
        ])

    def get_question(self, button):
        """Generate a question for the selected test part"""
        self.current_test_part = self.part_selector.value

        questions = {
            1: [
                "Tell me about your hometown. What do you like most about it?",
                "Do you work or study? What do you enjoy most about your work/studies?",
                "What are your hobbies? How long have you been interested in them?",
                "Describe your daily routine. What's your favorite part of the day?",
                "Tell me about your family. Are you close to your family members?"
            ],
            2: [
                "Describe a memorable event from your childhood. You should say: what the event was, when it happened, who was involved, and explain why it was memorable.",
                "Talk about a person who has influenced you. You should say: who this person is, how you know them, what influence they had on you, and explain why this person is important to you.",
                "Describe a place you would like to visit. You should say: where it is, what you can do there, what it looks like, and explain why you want to visit this place.",
                "Talk about a skill you would like to learn. You should say: what the skill is, why you want to learn it, how you would learn it, and explain how this skill would be useful to you."
            ],
            3: [
                "What role does technology play in modern education?",
                "How do you think social media affects relationships between people?",
                "What are the advantages and disadvantages of living in a big city?",
                "How important is it for people to learn about their cultural heritage?",
                "Do you think traditional skills are being lost in modern society? Why or why not?"
            ]
        }

        self.current_question = random.choice(questions[self.current_test_part])

        self.question_display.value = f"""
        <div style="background: #f0f8ff; padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>🎯 Your Question (Part {self.current_test_part}):</h3>
            <p style="font-size: 16px; font-weight: bold;">{self.current_question}</p>
            <p style="font-style: italic; color: #666;">
                {'Take a moment to think, then click "Start Recording" and speak for 1-2 minutes.' if self.current_test_part == 1
                else 'You have 1 minute to prepare, then speak for 1-2 minutes.' if self.current_test_part == 2
                else 'Think about your response and speak for 1-2 minutes.'}
            </p>
        </div>
        """

    def start_recording(self, button):
        """Start recording user's response"""
        self.start_recording_btn.disabled = True
        self.stop_recording_btn.disabled = False
        self.evaluate_btn.disabled = True

        with self.results_output:
            clear_output(wait=True)
            print("🎤 Recording started... Please allow microphone access if prompted.")
            print("Speak clearly into your microphone.")
            print("Click 'Stop Recording' when you're finished speaking.")

        # Start recording
        self.audio_recorder.start_recording()

    def stop_recording(self, button):
        """Stop recording user's response"""
        success = self.audio_recorder.stop_recording()

        self.start_recording_btn.disabled = False
        self.stop_recording_btn.disabled = True

        if success:
            self.evaluate_btn.disabled = False
            with self.results_output:
                clear_output(wait=True)
                print("🔴 Recording stopped!")
                print("Processing audio data...")
                print("Click 'Get Evaluation' to analyze your response.")
        else:
            with self.results_output:
                clear_output(wait=True)
                print("❌ Recording failed. Please try again.")

    def evaluate_response(self, button):
        """Evaluate the user's response"""
        self.evaluate_btn.disabled = True

        with self.results_output:
            clear_output()
            print("🔄 Processing your response...")

            # Save audio
            audio_file = self.audio_recorder.save_audio()
            if not audio_file:
                print("❌ No audio recorded. Please try recording again.")
                self.evaluate_btn.disabled = False
                return

            # Transcribe audio
            print("📝 Transcribing your speech...")
            transcription = self.evaluator.transcribe_audio(audio_file)
            if not transcription:
                print("❌ Failed to transcribe audio. Please try again.")
                self.evaluate_btn.disabled = False
                return

            print(f"✅ Transcription completed ({transcription['duration']:.1f}s)")

            # Analyze with AI
            print("🤖 Analyzing your response with AI...")
            analysis = self.evaluator.analyze_speech_with_ai(transcription['text'], self.current_test_part)
            if not analysis:
                print("❌ Failed to analyze response. Please try again.")
                self.evaluate_btn.disabled = False
                return

            # Display results
            self.display_evaluation_results(transcription, analysis)

            # Generate follow-up questions
            print("❓ Generating follow-up questions...")
            follow_ups = self.evaluator.generate_follow_up_questions(transcription['text'], self.current_test_part)
            self.display_follow_up_questions(follow_ups)

        self.evaluate_btn.disabled = False

    def display_evaluation_results(self, transcription, analysis):
        """Display comprehensive evaluation results"""
        with self.results_output:
            print("\n" + "="*60)
            print("📄 YOUR RESPONSE TRANSCRIPTION")
            print("="*60)
            print(f'"{transcription["text"]}"')
            print(f"\nDuration: {transcription['duration']:.1f} seconds")

            print("\n" + "="*60)
            print("🎯 IELTS SCORES")
            print("="*60)
            scores = analysis['scores']
            print(f"Overall Score: {scores['overall']}/9")
            print(f"├── Fluency & Coherence: {scores['fluency_coherence']}/9")
            print(f"├── Lexical Resource: {scores['lexical_resource']}/9")
            print(f"├── Grammatical Range: {scores['grammatical_range']}/9")
            print(f"└── Pronunciation: {scores['pronunciation']}/9")

            print(f"\n📊 Band Level: {analysis['band_descriptor']}")

            print("\n" + "="*60)
            print("📋 DETAILED ANALYSIS")
            print("="*60)

            criteria = ['fluency_coherence', 'lexical_resource', 'grammatical_range', 'pronunciation']
            criteria_names = ['Fluency & Coherence', 'Lexical Resource', 'Grammatical Range', 'Pronunciation']

            for criterion, name in zip(criteria, criteria_names):
                if criterion in analysis['detailed_analysis']:
                    details = analysis['detailed_analysis'][criterion]
                    print(f"\n{name.upper()}:")
                    print(f"Assessment: {details['assessment']}")
                    print("Strengths:")
                    for strength in details['strengths']:
                        print(f"  ✅ {strength}")
                    print("Areas for improvement:")
                    for weakness in details['weaknesses']:
                        print(f"  ⚠️ {weakness}")

            print("\n" + "="*60)
            print("💡 RECOMMENDATIONS")
            print("="*60)
            for i, rec in enumerate(analysis['recommendations'], 1):
                print(f"{i}. {rec}")

            self.plot_scores(scores)

    def display_follow_up_questions(self, questions):
        """Display follow-up questions"""
        with self.results_output:
            print("\n" + "="*60)
            print("❓ FOLLOW-UP QUESTIONS")
            print("="*60)
            print("Try answering these questions to practice further:")
            for i, question in enumerate(questions, 1):
                print(f"{i}. {question}")

    def plot_scores(self, scores):
        """Create a visualization of the scores"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

        # Bar chart of individual scores
        criteria = ['Fluency &\nCoherence', 'Lexical\nResource', 'Grammatical\nRange', 'Pronunciation']
        score_values = [scores['fluency_coherence'], scores['lexical_resource'],
                       scores['grammatical_range'], scores['pronunciation']]

        bars = ax1.bar(criteria, score_values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A'])
        ax1.set_ylim(0, 9)
        ax1.set_ylabel('Score')
        ax1.set_title('IELTS Speaking Scores by Criteria')
        ax1.grid(axis='y', alpha=0.3)

        # Add value labels on bars
        for bar, value in zip(bars, score_values):
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{value}', ha='center', va='bottom', fontweight='bold')

        # Pie chart showing overall performance
        overall_score = scores['overall']
        remaining = 9 - overall_score

        ax2.pie([overall_score, remaining], labels=['Achieved', 'Remaining'],
                colors=['#4ECDC4', '#E8E8E8'], startangle=90,
                autopct=lambda pct: f'{pct/100*9:.1f}' if pct > 0 else '')
        ax2.set_title(f'Overall Score: {overall_score}/9')

        plt.tight_layout()
        plt.show()

    def display(self):
        """Display the interface"""
        return self.interface

# Initialize the components
audio_recorder = AudioRecorder()
evaluator = IELTSRealTimeEvaluator(client)

# Initialize and display the interface
interface = IELTSInteractiveInterface(evaluator, audio_recorder)
interface.display()


✅ Successfully connected to OpenAI API!


🎤 Starting recording... Please allow microphone access if prompted.


<IPython.core.display.Javascript object>

🔴 Stopping recording...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

🎤 Starting recording... Please allow microphone access if prompted.


<IPython.core.display.Javascript object>

🔴 Stopping recording...


<IPython.core.display.Javascript object>